In [3]:
#importing the libraries
!pip install langchain
!pip install faiss-cpu
!pip install openai
!pip install llama_index

#or you can use 

%pip install langchain
%pip install faiss-cpu
%pip install openai
%pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.2/566.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3
  Attempting uninstall: typing-inspect
    Found existing installation: typing-inspect 0.9.0
    Uninstalling typing-inspect-0.9.0:
      Successfully uninstalled typing-inspect-0.9.0


In [4]:
# API Key 
import os
os.environ["OPENAI_API_KEY"] = 'Add your API Key here'

In [9]:
import pickle
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('/Users/tarakram/Documents/Chatbot/data').load_data()
print(documents)

# save the index in your directory, if you run again and again it is going to cost so much tokens

index = GPTVectorStoreIndex.from_documents(documents)
# Save the index
index.storage_context.persist('/data/')

[Document(id_='570f8bb5-7e98-487e-b67e-5ad0725631df', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='162b6c132fba0cde8b8cd975a8deed99f7195e762f8094b8fcb4c2b51cf852c4', text="Question: what does mysteryvibe do?\nAnswer: mysteryvibe is an award-winning world leader in medical devices for sexual health. we create clinically proven, fda class-2 devices that deliver precise medical frequency vibrations to address various sexual dysfunctions like genito-pelvic pain, penetration pain, arousal disorder, erectile dysfunction & prostate health.childbirth, menopause, diabetes, heart disease, cancer - all these life-events impact our sexual health. 50% men suffer from ed; 86% women suffer from genito-pain.we solve these through vibratory stimulation in a non-invasive way without side effects. our patented technology of malleable electronics adapts to the body and deliver targeted vibrations that increase blood flow to the affect

In [11]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='/Users/tarakram/Documents/Chatbot/index')
# load index
index = load_index_from_storage(storage_context)

In [12]:
index

In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the founder of Mystery vibe ?")
print(response)


The founder of Mysteryvibe is Dr. Soum Rakshit. He is a co-founder and CEO of the company and has a PhD and post-doc in biomedical engineering. He previously created a start-up in eye-recognition which was acquired by a US defence company. He then spent 7 years in management consulting on large technology projects before starting Mysteryvibe in 2014.


In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("what is Mystery Vibe do?")
print(response)


MysteryVibe is an award-winning world leader in medical devices for sexual health. We create clinically proven, FDA class-2 devices that deliver precise medical frequency vibrations to address various sexual dysfunctions like genito-pelvic pain, penetration pain, arousal disorder, erectile dysfunction & prostate health. Our patented technology of malleable electronics adapts to the body and deliver targeted vibrations that increase blood flow to the affected areas & changes the message carried by the nerves. Our devices have changed the lives of 100k patients, won 30 international awards, 35 patents & design registrations and published 3 papers in the journal of sexual medicine. They are recommended by Mayo Clinic, Cedar-Sinai, NHS, Charité & others.


In [20]:
# Chat Bot

import openai
import json

class Chatbot:
    def __init__(self, api_key, index):
        self.index = index
        openai.api_key = api_key
        self.chat_history = []

    def generate_response(self, user_input):
        prompt = "\n".join([f"{message['role']}: {message['content']}" for message in self.chat_history[-5:]])
        prompt += f"\nUser: {user_input}"
        query_engine = index.as_query_engine()
        response = query_engine.query(user_input)

        message = {"role": "assistant", "content": response.response}
        self.chat_history.append({"role": "user", "content": user_input})
        self.chat_history.append(message)
        return message

    def load_chat_history(self, filename):
        try:
            with open(filename, 'r') as f:
                self.chat_history = json.load(f)
        except FileNotFoundError:
            pass

    def save_chat_history(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.chat_history, f)

In [23]:
# Swap out your index below for whatever knowledge base you want
bot = Chatbot("Add your API Key here ", index=index)
bot.load_chat_history("chat_history.json")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        print("Bot: Goodbye!")
        bot.save_chat_history("chat_history.json")
        break
    response = bot.generate_response(user_input)
    print(f"Bot: {response['content']}")

You: Hey how are you ?
Bot: 
Answer: Hi there, I'm doing well, thanks for asking. How about you?
You: Who is the founder of the Mystery vibe ?
Bot: 
The founder of Mysteryvibe is Dr. Soum Rakshit. He is a biomedical engineer and entrepreneur with a PhD and post-doc in biomedical engineering. He previously created a start-up in eye-recognition which was acquired by a US defence company. In 2014, Soum started Mysteryvibe to bring defence electronics and medical expertise together and create devices that solve big areas in sexual health in an accessible way.
You: Great !, please tell me more about the company and what it is ?
Bot: 
MysteryVibe is a sexual health technology company that creates products to improve people's lives. Founded in 2014 by Dr. Soum Rakshit, MysteryVibe is on a mission to normalize sexual health and make solutions for major sexual health issues accessible to all. The company has created 6 FDA Class 2 medical devices addressing genito-pelvic pain, penetration pain, 